# Incorporation of heterologous genes in the iYO844 model for DHAA production 

In [1]:
# Loading the required packages
from cobra.io import read_sbml_model, write_sbml_model # To read the model and save new newly generated model
from cobra import Reaction, Metabolite # To create new reactions and metabolites
from cameo.strain_design import pathway_prediction # To predict if there exist a pathway for a given metabolite

In [2]:
# Loading the the model
iYO844 = read_sbml_model("data/iYO844.xml")
# Generating a copy of iY0844 in a variable to add the reactions to the new model
iYO844_modified = iYO844.copy() 
iYO844

Name,iYO844
Memory address,0x07f7e184adf50
Number of metabolites,990
Number of reactions,1250
Number of groups,0
Objective expression,1.0*BIOMASS_BS_10 - 1.0*BIOMASS_BS_10_reverse_8788b
Compartments,"cytosol, extracellular space"


First we make sure, that the MEP pathway is present in the current model iY40844 by searching for a pathway for creation of geranyl diphosphate, the second last step of the pathway.

In [3]:
# Might take a long time to run! 3-4 min
# Predicting if there already exists a patwhay for the end product
predictor = pathway_prediction.PathwayPredictor(iYO844) # Loading the model into the predictor tool
# Specifying metabolite. From KEGG we know only two known pathways are found.
pathways = predictor.run(product='geranyl diphosphate', max_predictions=2) 


,equation,lower_bound,upper_bound
MNXR69185,geranyl diphosphate + isopentenyl diphosphate ...,-1000.0,1000.0


,equation,lower_bound,upper_bound
MNXR55238,geranyl diphosphate + diphosphate <=> dimethyl...,-1000.0,1000.0


Here it can be seen that the precursor geranyl diphosphate is produced in two reactions. From litterature, we know that  isopentenyl pyrophosphate and dimethylallyl pyrophosphate (in the model called ipdp_c and dmpp_c, respectively) are isomers of each other and the isomerisation reaction occurs in the step before formation of geranyl diphosphate, hence the two reactions above. Geranyl diphosphate is therefor only produced through one pathway.

### First reaction incorporation
GPP --> FPP \
Geranyl diphosphate + Isopentenyl diphosphate <=> Diphosphate + trans,trans-Farnesyl diphosphate \
Catalysed by farnesyl pyrophosphate synthase (FPPS) \
KEGG reaction: https://www.genome.jp/entry/R02003

In [4]:
# Creating the reaction
reaction_1 = Reaction('FPPS') # FPPS: farnesyl pyrophosphate synthase

# Adding the metabolites based on the stoichiometric reaction
reaction_1.add_metabolites({iYO844.metabolites.get_by_id('grdp_c'): -1,
                            iYO844.metabolites.get_by_id('ipdp_c'): -1,
                            iYO844.metabolites.get_by_id('ppi_c'): 1,
                            iYO844.metabolites.get_by_id('frdp_c'): 1
                            })

print(reaction_1.build_reaction_string()) # Printing the reaction to check
iYO844_modified.add_reactions([reaction_1]) # Adding the reaction to the new iY0844 model

grdp_c + ipdp_c --> frdp_c + ppi_c


### Second reaction incorporation
FPP --> amorphadiene / amorpha-4,11-diene + Diphosphate \
Catalysed by amorphadiene synthase (ADS) \
KEGG reaction: https://www.genome.jp/entry/R07630

In [5]:
# Creating the reaction
reaction_2 = Reaction('ADS') # ADS: amorphadiene synthase

# Defining the new metabolite, as it does not exists in GSM
amorphadiene = Metabolite(id='amorph_c', name='amorphadiene', compartment='c', formula='C15H24')

# Adding the metabolites based on the stoichiometric reaction
reaction_2.add_metabolites({iYO844.metabolites.get_by_id('frdp_c'): -1,
                            amorphadiene: 1,
                            iYO844.metabolites.get_by_id('ppi_c'): 1})
print(reaction_2.build_reaction_string()) # Printing the reaction to check
iYO844_modified.add_reactions([reaction_2]) # Adding the reaction to the new iY0844 model

frdp_c --> amorph_c + ppi_c


### Third reaction incorporation
Amorpha-4,11-diene + Oxygen + [Reduced NADPH---hemoprotein reductase] <=> Artemisinic alcohol + [Oxidized NADPH---hemoprotein reductase] + H2O \
KEGG reaction: https://www.kegg.jp/entry/R10023

[Reduced NADPH---hemoprotein reductase] means: NADP+ + n reduced hemoprotein \
[Oxidized NADPH---hemoprotein reductase] means: NADPH + H+ + n oxidized hemoprotein \
https://en.wikipedia.org/wiki/NADPH—hemoprotein_reductase


In [6]:
# Creating the reaction
reaction_3 = Reaction('CYP71AV1') # CYP71AV1: amorphadiene oxidase

# Defining the new metabolite, as it does not exists in GSM
artemisinic_alcohol = Metabolite(id='artalc_c', name='artemisinic_alcohol', compartment='c', formula='C15H24O') 

# Adding the metabolites based on the stoichiometric reaction
reaction_3.add_metabolites({amorphadiene: -1,
                            iYO844.metabolites.get_by_id('o2_c'): -1,
                            iYO844.metabolites.get_by_id('nadp_c'): -1,
                            artemisinic_alcohol: 1, 
                            iYO844.metabolites.get_by_id('nadph_c'): 1,
                            iYO844.metabolites.get_by_id('h2o_c'): 1})

print(reaction_3.build_reaction_string()) # Printing the reaction to check
iYO844_modified.add_reactions([reaction_3]) # Adding the reaction to the new iY0844 model

amorph_c + nadp_c + o2_c --> artalc_c + h2o_c + nadph_c


### Fourth reaction incorporation
Artemisinic alcohol + Oxygen + [Reduced NADPH---hemoprotein reductase] <=> Artemisinic aldehyde + [Oxidized NADPH---hemoprotein reductase] + 2 H2O \
KEGG reaction: https://www.genome.jp/entry/R10024


In [7]:
# Creating the reaction
reaction_4 = Reaction('ADH1_CYP71AV1') # ADH1: alcohol dehydrogenase, CYP71AV1: amorphadiene oxidase

# Defining the new metabolite, as it does not exists in GSM
artemisinic_aldehyde = Metabolite(id='artald_c', name='artemisinic aldehyde', compartment='c', formula='C15H22O')

# Adding the metabolites based on the stoichiometric reaction
reaction_4.add_metabolites({artemisinic_alcohol: -1,
                            iYO844.metabolites.get_by_id('o2_c'): -1,
                            iYO844.metabolites.get_by_id('nadp_c'): -1,
                            artemisinic_aldehyde: 1, 
                            iYO844.metabolites.get_by_id('nadph_c'): 1,
                            iYO844.metabolites.get_by_id('h2o_c'): 2})

print(reaction_4.build_reaction_string()) # Printing the reaction to check
iYO844_modified.add_reactions([reaction_4]) # Adding the reaction to the new iY0844 model

artalc_c + nadp_c + o2_c --> artald_c + 2 h2o_c + nadph_c


### Fifth reaction incorporation
Artemisinic aldehyde + NADPH + H+  <=> (11R)-Dihydroartemisinic aldehyde + NADP+ \
KEGG reaction: https://www.genome.jp/entry/R10075

In [8]:
# Creating the reaction
reaction_5 = Reaction('DBR2') # DBR2: artemisinic aldehyde double-bond reductase

# Defining the new metabolite, as it does not exists in GSM
dihydroartemisinic_aldehyde = Metabolite(id='dihydroartald_c', name='dihydroartemisinic aldehyde', compartment='c', formula='C15H24O')

# Adding the metabolites based on the stoichiometric reaction
reaction_5.add_metabolites({artemisinic_aldehyde: -1,
                            iYO844.metabolites.get_by_id('nadph_c'): -1,
                            iYO844.metabolites.get_by_id('h_c'): -1,
                            dihydroartemisinic_aldehyde: 1, 
                            iYO844.metabolites.get_by_id('nadp_c'): 1})

print(reaction_5.build_reaction_string()) # Printing the reaction to check
iYO844_modified.add_reactions([reaction_5]) # Adding the reaction to the new iY0844 model

artald_c + h_c + nadph_c --> dihydroartald_c + nadp_c


### Sixth reaction incorporation
Dihydroartemisinic aldehyde + Oxygen + [Reduced NADPH---hemoprotein reductase] <=> Dihydroartemisinic acid + [Oxidized NADPH---hemoprotein reductase] + H2O

Modified from KEGG reaction: https://www.genome.jp/entry/R10025


In [9]:
# Creating the reaction
reaction_6 = Reaction('ALDH1_CYP71AV1') # ALDH1: aldehyde dehydrogenase 1, CYP71AV1: amorphadiene oxidase

# Defining the new metabolite, as it does not exists in GSM
dihydroartemisinic_acid = Metabolite(id='DHAA_c', name='dihydroartemisinic acid', compartment='c', formula='C15H24O2')

# Adding the metabolites based on the stoichiometric reaction
reaction_6.add_metabolites({artemisinic_alcohol: -1,
                            iYO844.metabolites.get_by_id('o2_c'): -1,
                            iYO844.metabolites.get_by_id('nadp_c'): -1,
                            dihydroartemisinic_acid: 1, 
                            iYO844.metabolites.get_by_id('nadph_c'): 1,
                            iYO844.metabolites.get_by_id('h2o_c'): 1})

print(reaction_6.build_reaction_string()) # Printing the reaction to check
iYO844_modified.add_reactions([reaction_6]) # Adding the reaction to the new iY0844 model

artalc_c + nadp_c + o2_c --> DHAA_c + h2o_c + nadph_c


### Seventh reaction incorporation
Artemisinic aldehyde + Oxygen + [Reduced NADPH---hemoprotein reductase] <=> Artemisinate + [Oxidized NADPH---hemoprotein reductase] + H2O \
KEGG reaction: https://www.genome.jp/entry/R10025

[Reduced NADPH---hemoprotein reductase] means: NADP+ + n reduced hemoprotein \
[Oxidized NADPH---hemoprotein reductase] means: NADPH + H+ + n oxidized hemoprotein

In [10]:
# Creating the reaction
reaction_7 = Reaction('ALDH1_CYP71AV1_2') # ALDH1: aldehyde dehydrogenase 1, CYP71AV1: amorphadiene oxidase

# Defining the new metabolite, as it does not exists in GSM
artemisinic_acid = Metabolite(id='artac_c', name='artemisinic acid', compartment='c', formula='C15H22O2')

# Adding the metabolites based on the stoichiometric reaction
reaction_7.add_metabolites({artemisinic_alcohol: -1,
                            iYO844.metabolites.get_by_id('o2_c'): -1,
                            iYO844.metabolites.get_by_id('nadp_c'): -1,
                            artemisinic_acid: 1, 
                            iYO844.metabolites.get_by_id('nadph_c'): 1,
                            iYO844.metabolites.get_by_id('h2o_c'): 1})

print(reaction_7.build_reaction_string()) # Printing the reaction to check
iYO844_modified.add_reactions([reaction_7]) # Adding the reaction to the new iY0844 model

artalc_c + nadp_c + o2_c --> artac_c + h2o_c + nadph_c


### Testing implementation

In [11]:
# Fetching the artimisinin precursor to check it exists in the model
iYO844_modified.metabolites.DHAA_c

Metabolite identifier,DHAA_c
Name,dihydroartemisinic acid
Memory address,0x07f7e091ef310
Formula,C15H24O2
Compartment,c
In 1 reaction(s),ALDH1_CYP71AV1


In [12]:
# Specifying which metabolite, we want to test the production of
DHAA = iYO844_modified.add_boundary(iYO844_modified.metabolites.get_by_id('DHAA_c'), type='demand')

# Testing production of DHAA in the system after pathway implementation in model to ensure metabolic connectivity
with iYO844_modified :
    iYO844_modified.objective = DHAA #changing objective of GSM to be production of DHAA
    sol = iYO844_modified.optimize()
    print(sol.objective_value)

0.36342857142857143


### Saving the model

In [13]:
# Saving the variable as a new xml file
iYO844_modified.id = "iYO844_modified"
write_sbml_model(iYO844_modified, 'data/iYO844_modified.xml')

### Memote analysis on modified model

In [14]:
model = read_sbml_model("data/iYO844_modified.xml")
model

Name,iYO844_modified
Memory address,0x07f7e091ef6d0
Number of metabolites,996
Number of reactions,1258
Number of groups,0
Objective expression,1.0*BIOMASS_BS_10 - 1.0*BIOMASS_BS_10_reverse_8788b
Compartments,"cytosol, extracellular space"


In [15]:
# Generating memote analysis and saving result in html file
!memote report snapshot data/iYO844_modified.xml --filename memote/iYO844_modified_memote.html

The current solver interface glpk doesn't support setting the optimality tolerance.
============================= test session starts ==============================
platform darwin -- Python 3.7.16, pytest-4.6.11, py-1.11.0, pluggy-0.13.1
rootdir: /Users/millegjedingskovsorensen
plugins: anyio-3.7.1
collected 146 items / 1 skipped / 145 selected                                 

../../../anaconda3/envs/cobra_escher_final/lib/python3.7/site-packages/memote/suite/tests/test_annotation.py F [  0%]
F.FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF..FF..F...F....FFF.FFFFFFFFFF..         [ 44%]
../../../anaconda3/envs/cobra_escher_final/lib/python3.7/site-packages/memote/suite/tests/test_basic.py . [ 45%]
.....F........F...F.FF                                                   [ 60%]
../../../anaconda3/envs/cobra_escher_final/lib/python3.7/site-packages/memote/suite/tests/test_biomass.py . [ 60%]
F.....F.F                                                                [ 67%]
../../../anaconda3/envs/cobra_es